# Sample Design App
## This app will allow you to create random, systematic, and stratified random sample designs.
### Steps
1. Draw a polygon on the map that spatially identifies the boundary of the study unit.
2. Select the sample type and design values.
4. Click the run button. Sample point locations will be added to the map and will be zipped up for download. 
### Sampling strategies
- random - simple random sample within the boundary of your geometry (n=sample size)
- systematic - systematic random sample with locations spread based on Distance x and Distance y (n = total number of locations that can fit into the polygon)
- tiled - a simple random sample within a defined tiled area that makes sure locations are no closer than a user user defined amount. Tiles are built using the square root of the area parameter (n= number of locations within each tile).
- stratified - a stratified random sample that uses remotely sensed data to define k-mean clusters and randomly allocate locations within each cluster. In addition to defining remotely sensed data users can specify if they want a proportional or equal number of locations within each k-mean class and if users want locations to be balanced towards cluster centers or randomly selected. Remotely sensed values, clusters labels, cluster distance, and cluster weights will be added to the attribute table. Big N is used to create a large simple random sample to select n locations from (n = sample size). 

In [ ]:
import smpDsg
import ipyleaflet
import ipywidgets
import geopandas as gpd
import shapely
import base64
import os
from raster_tools import Raster, zonal, surface, general

if('LOCALTILESERVER_CLIENT_PREFIX' in os.environ.keys()):
    os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = f"{os.environ['JUPYTERHUB_SERVICE_PREFIX']}/proxy/{{port}}"

lc = ipyleaflet.LayersControl()
lc.position='topright'
dr=ipyleaflet.DrawControl()
dr.position='topleft'


bmaps=ipyleaflet.basemaps

m=ipyleaflet.Map(controls=[lc,dr],scroll_wheel_zoom=True,center=(41,-119),zoom=5,layout=ipywidgets.Layout(height='90%'))

tly1=ipyleaflet.basemap_to_tiles(bmaps.Esri.WorldImagery)
tly1.base = False
tly1.name = 'World Imagery'

tly2=ipyleaflet.ImageService(
    url=r'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF230/US_230EVT/ImageServer',
    format='jpgpng',
    attribution='USDA USFS',
    name = 'Landfire EVT',
)


tly3=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Nat_BurnProbability/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name = 'Burn Probability'
)

tly4=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Potential_Control_Location/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name='PCL',
)
tly5=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/SnagHazard_eDart/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name='Snag Hazard eDart',
)
tly6=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Suppression_Difficulty_Index_80/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name='SDI 80',

)

baselyrDic = {
    'World Imagery':tly1,
    'Landfire EVT': tly2,
    'Burn Probability': tly3,
    'PCL': tly4,
    'Snag Hazard eDart':tly5,
    'SDI 80':tly6,
}

def add_remove_layer(b):
    nm = b['owner'].description
    lyr = baselyrDic[nm]
    if b['new']:
        m.add(lyr)
    else:
        m.remove(lyr)


#widgets
style = {'description_width': 'initial'}

#optional background layers
l1_o=ipywidgets.widgets.Checkbox(value=False,description='World Imagery', disabled=False, indent=False)
l1_o.observe(add_remove_layer,names='value')
l2_o=ipywidgets.widgets.Checkbox(value=False,description='Landfire EVT', disabled=False, indent=False)
l2_o.observe(add_remove_layer,names='value')
l3_o=ipywidgets.widgets.Checkbox(value=False,description='Burn Probability', disabled=False, indent=False)
l3_o.observe(add_remove_layer,names='value')
l4_o=ipywidgets.widgets.Checkbox(value=False,description='PCL', disabled=False, indent=False)
l4_o.observe(add_remove_layer,names='value')
l5_o=ipywidgets.widgets.Checkbox(value=False,description='Snag Hazard eDart', disabled=False, indent=False)
l5_o.observe(add_remove_layer,names='value')
l6_o=ipywidgets.widgets.Checkbox(value=False,description='SDI 80', disabled=False, indent=False)
l6_o.observe(add_remove_layer,names='value')

thb=ipywidgets.HBox([l1_o,l2_o,l3_o,l4_o,l5_o,l6_o])  

lbl=ipywidgets.Label(value='Add/Remove Background Layers')
lbl.style.font_size="20px"
thb=ipywidgets.VBox([lbl,thb])

def add_widgets(b):
    vl=b['new']
    if(vl=='systematic'):
        wb.children=tuple(list(wb.children[:3]) +[smp_spcx,smp_spcy,btn,outlnks])
    elif(vl=='tiled'):
        wb.children=tuple(list(wb.children[:3]) +[smp_pnt,smp_spca, smp_spcd, btn,outlnks])
    elif(vl=='stratified'):
        wb.children=tuple(list(wb.children[:3]) +[smp_bn,smp_pnt,smp_rsp,smp_kc,smp_prop,smp_cent,smp_opt_lbl,smp_date1,smp_date2,btn,outlnks])
    else:
        wb.children=tuple(list(wb.children[:3]) +[smp_pnt,btn,outlnks])


#widgets
style = {
    'description_width': 'initial',
    'width': 'max-content',
    }

#main widgets
smp_dsg=ipywidgets.widgets.Dropdown(
    options=['random', 'systematic','tiled','stratified'],
    value='random',
    description='Sample Type:',
    disabled=False,
    style=style,
)
smp_dsg.observe(add_widgets,names='value')

smp_pnt=ipywidgets.widgets.IntText(
    value=30,
    description='Sample Size',
    disable=False,
    style=style,
)
smp_bn=ipywidgets.widgets.IntText(
    value=10000,
    description='Big N',
    disable=False,
    style=style,
)
smp_spcx=ipywidgets.widgets.IntText(
    value=100,
    description='Distance x (m)',
    disable=False,
    style=style,
)
smp_spcy=ipywidgets.widgets.IntText(
    value=100,
    description='Distance y (m)',
    disable=False,
    style=style,

)

smp_spcd=ipywidgets.widgets.IntText(
    value=100,
    description='Minimum Distance (m)',
    disable=False,
    style=style,

)

smp_spca=ipywidgets.widgets.IntText(
    value=500*500,
    description='Area (square meters)',
    disable=False,
    style=style,

)

smp_rsp=ipywidgets.widgets.SelectMultiple(
    options=['elevation', 'sentinel2','landsat'],
    value=['elevation'],
    description='K-means sources',
    disabled=False,

    style=style,
)

smp_opt_lbl=ipywidgets.widgets.Label(value='Dates only apply to Imagery', style=style,)

smp_date1=ipywidgets.widgets.DatePicker(
    description='Start date',
    disable=False,
    style=style,
)

smp_date2=ipywidgets.widgets.DatePicker(
    description='End date',
    disable=False,
    style=style,
)

smp_kc=ipywidgets.widgets.IntText(
    value=10,
    description='K Classes',
    disable=False,
    style=style,
)

smp_prop=ipywidgets.widgets.Checkbox(
    value=True,
    description='Proportional',
    disable=False,
    style=style,
)

smp_cent=ipywidgets.widgets.Checkbox(
    value=False,
    description='Center',
    disable=False,
    style=style,
)

prg_wdg=ipywidgets.widgets.IntProgress(value=0,min=0,max=12,description='Progress',style={'bar_color':'green'},orientation='horizontal')
out_wdg=ipywidgets.widgets.Output(layout={'border': '1px solid black'})

def create_download_link(filename, title = "Click here to download: "):  
    fl = open(filename, "rb")
    data = fl.read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    fl.close()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank" style="color:blue">{title}</a>'
    html = html.format(payload=payload,title=title+f' {filename}',filename=filename)
    return html

cnt=1

@out_wdg.capture()
def _run_sample_design(b):
    global cnt, out_wdg
    out_wdg.clear_output()
    def _get_dt_str(dt):
        dy=str(dt.day)
        if(len(dy)<2):dy='0'+dy
        mth = str(dt.month)
        if(len(mth)<2):mth='0'+mth
        yr=str(dt.year)
        return yr+'-'+mth+'-'+dy

    polylst=[]
    for d in dr.data:
        geo=d['geometry']
        t=geo['type']
        if(t=='Polygon'):polylst.append(geo['coordinates'][0])

    if((len(polylst)<1)):
        print("You Must Create a Study Area afore clicking the Run button")
        return
    else:
        geo=gpd.GeoSeries(shapely.polygons(polylst),crs=4326)
        geop=geo.to_crs(5070)
        xmin,ymin,xmax,ymax=geop.buffer(100).total_bounds
        print('Projected to Albers Equal Area ...')
        print('Creating Sample ...')
        prg_wdg.value=5

        gdfp=None
        if(smp_dsg.value=='systematic'):
            gdfp=smpDsg.get_systematic_sample(geop,smp_spcx.value,smp_spcy.value,) 
        elif(smp_dsg.value=='random'):
            gdfp=smpDsg.get_random_sample(geop,smp_pnt.value)
        elif(smp_dsg.value=='tiled'):
            gdfp=smpDsg.get_spread_pnts(geop,n=smp_pnt.value,min_dist=smp_spcd.value,area=smp_spca.value)

        elif(smp_dsg.value=='stratified'):
            tdfp=smpDsg.get_random_sample(geom_p=geop,n=smp_bn.value)
            url="https://planetarycomputer.microsoft.com/api/stac/v1"
            bdt=smp_date1.value
            edt=smp_date2.value
            if(bdt is None or edt is None):
                dtstr=''
            else:
                dtstr=_get_dt_str(bdt)+'/'+_get_dt_str(edt)

            for s in smp_rsp.value:
                prg_wdg.value=prg_wdg.value+1
                print('Getting ' + s +' data ...')
                limit=1000
                if s=='elevation': 
                    name='cop-dem-glo-30'
                    xra,ic=smpDsg.get_stac_data(geo[0],url=url,name=name,res=30,crs=5070,limit=1000)
                    elv=Raster(xra.sel(x=slice(xmin,xmax),y=slice(ymax,ymin)))
                    slp=surface.slope(elv)
                    nrt=surface.northing(elv)
                    est=surface.easting(elv)
                    curv=surface.curvature(elv)
                    rs=general.band_concat([elv,slp,nrt,est,curv])
                elif(s=='sentinel2'):
                    name= "sentinel-2-l2a"
                    query={'eo:cloud_cover':{'lt':10}}
                    xra,ic=smpDsg.get_stac_data(geo[0],url=url,name=name,res=30,crs=5070,query=query,datetime=dtstr,limit=1000)
                    if(xra is None): continue
                    rs=Raster(xra.sel(band=['B02','B03','B04','B08'],x=slice(xmin,xmax),y=slice(ymax,ymin)))
                elif s=='landsat':
                    name='landsat-c2-l2'
                    query={'eo:cloud_cover':{'lt':10},'platform':{'eq':'landsat-8'}}
                    xra,ic=smpDsg.get_stac_data(geo[0],url=url,name=name,res=30,crs=5070,query=query,datetime=dtstr,limit=1000)
                    if(xra is None): continue
                    rs=Raster(xra.sel(band=['blue','green','red','nir08','lwir11', 'swir16', 'swir22'],x=slice(xmin,xmax),y=slice(ymax,ymin)))
                else:
                    pass
                
                
                print('Exacting cell values ...')

                tbl=zonal.extract_points_eager(tdfp,rs,s[:3],axis=1).compute()
                pred=tbl.columns.values
                tdfp=tdfp.join(tbl)
                
            print('Selecting locations ...')
            gdfp=smpDsg.get_spread_balanced_sample_subarea(tdfp,pred,k=smp_kc.value,n=smp_pnt.value,equal_n_classes=not smp_prop.value,center=smp_cent.value)
            prg_wdg.value=prg_wdg.value+1

            
        else:
            gdfp=smpDsg.get_random_sample(geop,smp_pnt.value)
        
        gdfg=gdfp.to_crs(4326)
        fnm=smp_dsg.value + 'sample ' + str(cnt)
        print('Adding sample to the map ...')
        prg_wdg.value=10

        lyr=ipyleaflet.GeoData(geo_dataframe=gdfg,
                               style={'color': 'yellow', 'radius':4, 'fillColor': 'yellow', 'opacity':1, 'weight':2, 'dashArray':'2', 'fillOpacity':1},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               point_style={'radius': 4, 'color': 'yellow', 'fillOpacity': 1, 'fillColor': 'yellow', 'weight': 3},
                               name = fnm)
        
        print('Creating download file ...')
        gdfp.to_file(fnm + '.shp.zip',driver='ESRI Shapefile')
        html=create_download_link(fnm+'.shp.zip')
        outlnks.children=tuple(list(outlnks.children) +[ipywidgets.HTML(html)])

        m.add(lyr)
        cnt+=1
        prg_wdg.value=prg_wdg.max
        print('Finished all processing ...')

        return gdfp


def sample_design(b):
    prg_wdg.value=0
     
    #check values
    gdf=_run_sample_design(b)
        
    return gdf

  
btn=ipywidgets.widgets.Button(description='Run')
gdf=btn.on_click(_run_sample_design)

outlnks=ipywidgets.VBox([])


wb=ipywidgets.VBox([ipywidgets.widgets.Label(value=""),ipywidgets.widgets.Label(value=""),smp_dsg,smp_pnt,btn,outlnks],layout=ipywidgets.Layout(width='30%'))

rb=ipywidgets.VBox([thb,m],layout=ipywidgets.Layout(width='70%',height='800px'))

mb=ipywidgets.HBox([wb,rb])

ipywidgets.VBox([mb,prg_wdg,out_wdg])